In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [ ]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2-9b-it.gguf", device_map="tpu:0",
                                         from_type="gemma2",
                                         load_eager=True
                                         )

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("UCLA-AGI/Gemma-2-9B-It-SPPO-Iter3")
tokenizer.padding_side = "right"

In [4]:
# import jax.numpy as jnp
# import jax
# from micrlhf.llama import LlamaBlock

# tokens = tokenizer.encode("The quick brown fox jumps over the lazy dog")
# get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
#     pz.nn.Sequential([
#         pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
#         x
#     ])
# )
# get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
# token_array = jnp.asarray([tokens] * 4)
# token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
# token_array = pz.nx.wrap(token_array, "batch", "seq")
# inputs = llama.inputs.from_basic_segments(token_array)
# get_resids(inputs)[1][0].value.unwrap("batch", "seq", "embedding")[0]

In [5]:
from micrlhf.sampling import sample
chat = [
    { "role": "user", "content": "Write a hello world program" },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
sample(llama, tokenizer, prompt,
       batch_size=4, do_sample=True)[0]

  0%|          | 0/49 [00:00<?, ?it/s]

In [ ]:
# sample(llama, tokenizer, ["Write me a poem about Machine Learning.", "Hi!"], do_sample=True, return_only_completion=True, seed=3)

In [ ]:
# from transformers import AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained("alpindale/gemma-2b")

In [ ]:
# import torch
# from collections import OrderedDict
# for m in model.modules():
#     m._forward_hooks = OrderedDict()
# model.model.layers[0].register_forward_hook(lambda self, input, output: print(input[0]))
# tt = torch.LongTensor(tokens).unsqueeze(0)
# model(tt).logits